<a href="https://colab.research.google.com/github/AbhinavKoul/Pothole-Detection-Using-Convolutional-Neural-Networks/blob/Backend/Pothole_using_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
import os
os.chdir("/content/drive/My Drive/Potholes Dataset")
!ls

'Dataset 1 (Simplex)'   README.txt	     'Subset 2 (Complex)'
'Dataset 2 (Complex)'  'Subset 1 (Simplex)'


REACHED REQUIRED DIRECTORY.

In [0]:
#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import math
import random #rng
from PIL import Image #handle images
import matplotlib.pyplot as plt #plot data
from matplotlib.patches import Polygon #draw polygons on plots
import re #regex
from subprocess import check_output #run commands
%tensorflow_version 1.x
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, Convolution2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils

In [8]:
class ImageData:
    def __init__(self, img_name, boxes):
        self.img_name = img_name
        self.boxes = boxes

def parse_directory(path):
    """Parse a directory containing negative images to a list of ImageData"""
    img_names = check_output(["ls", path]).decode("utf8").split("\n")
    data = []
    for img_name in img_names:
        match = re.search(r'G\d+\.JPG', img_name)
        if(match):
            data.append(ImageData(match.group(), []))
    return data
    
def parse_txt_file(filepath, filename):
    """Parse the txt file containing image relative paths, name, and boxes data to a list of ImageData"""
    file = open(filepath + filename)
    items = file.read().strip().split("\n")
    item_regex = r'(^[\w\s]+\\(?:[\w\s]+\\)*G\d+\.bmp)\s+(\d+)\s+((?:\d+\s+\d+\s+\d+\s+\d+\s*)+)'
    
    def parse_item(item):
        match = re.match(item_regex, item)
        
        img_path = match.group(1).replace('.bmp', '.JPG') #txt says bmp but images are actually jpg
        img_name = re.search(r'G\d+\.JPG', img_path).group()
        num_boxes = int(match.group(2))
        values = re.findall(r'\d+', match.group(3))
        boxes = [tuple([ int(values[i*4+j]) for j in range(0,4)]) for i in range(0,num_boxes)]
        
        return ImageData(img_name, boxes)
    
    return [parse_item(item) for item in items]


class ImageColleciton:
    def __init__(self, data, path):
        self.data = data
        self.path = path
        
src_path = "/content/drive/My Drive/Potholes Dataset/"    # file location
dataset2_path = src_path + "Dataset 2 (Complex)/"
dataset1_path = src_path + "Dataset 1 (Simplex)/"

img_collections = {
    "dataset2_complex": {
    #     "train": {
    #         "positive": ImageColleciton(
    #             parse_txt_file(dataset2_path, "complexTrainFullSizeAllPotholes.txt"),
    #             dataset2_path + "Train data/Positive data/"
    #         ),
    #         "negative": ImageColleciton(
    #             parse_directory(dataset2_path + "Train data/Negative data/"),
    #             dataset2_path + "Train data/Negative data/"
    #         )
    #     },
        "test": ImageColleciton( #test images have only positive data
            parse_txt_file(dataset2_path, "complexTestFullSizeAllPotholes.txt"),
            dataset2_path + "Test data/"
        ),
    },
    "dataset1_simplex": {
        # "train": {
        #     "positive": ImageColleciton(
        #         parse_txt_file(dataset1_path, "simpleTrainFullPhotosSortedFullAnnotations.txt"),
        #         dataset1_path + "Train data/Positive data/"
        #     ),
        #     "negative": ImageColleciton(
        #         parse_directory(dataset1_path + "Train data/Negative data/"),
        #         dataset1_path + "Train data/Negative data/"
        #     )
        # },
        "test": ImageColleciton( #test images have only positive data
            parse_txt_file(dataset1_path, "simpleTestFullSizeAllPotholesSortedFullAnnotation.txt"),
            dataset1_path + "Test data/"
        ),
    }
}

print("done " + str(random.random()))

done 0.1497424855963193
